In [1]:
import requests
import re
import json
import numpy as np
import pickle
from pandas.io.json import json_normalize

**1). Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)**

In [2]:
def make_json(link):
    req = requests.get(link)
    data = json.loads(req.text)
    return data    

In [3]:
def find_tickets(from_city, to_city, one_way=True):
    city2iata_link = u'https://www.travelpayouts.com/widgets_suggest_params?'
    city2iata_link += u'q=Из%20{}%20в%20{}'.format(from_city, to_city)
    
    city2iata_data = make_json(city2iata_link)
    
    to_iata = city2iata_data['destination']['iata']
    from_iata = city2iata_data['origin']['iata']
    
    tickets_link = 'http://min-prices.aviasales.ru/calendar_preload?'
    tickets_link += 'origin={}&destination={}&one_way={}'.format(from_iata,
                                                                 to_iata,
                                                                 one_way)
    tickets_data = make_json(tickets_link)
    
    df = json_normalize(tickets_data['best_prices'])
    columns = ['depart_date', 'gate', 'origin', 'destination', 'return_date', 'value']
    if one_way:
        columns.remove('return_date')        
    df = df[columns]
    df['destination'] = city2iata_data['destination']['name']
    df['origin'] = city2iata_data['origin']['name']
    df.sort_values(by=['depart_date', 'value'], inplace=True)
    
    return df.set_index('depart_date')

In [4]:
tickets = find_tickets(u'Москва', u'Санкт-Петербург', one_way=False)

tickets.head(15)

,gate,origin,destination,return_date,value
depart_date,,,,,
2019-08-13,Aviakassa,Москва,Санкт-Петербург,2020-01-15,4555.0
2019-08-14,Pobeda,Москва,Санкт-Петербург,2019-11-14,4199.0
2019-08-15,Aviakassa,Москва,Санкт-Петербург,2019-11-15,4785.0
2019-08-16,KupiBilet.ru,Москва,Санкт-Петербург,2019-09-14,4930.0
2019-08-17,Pobeda,Москва,Санкт-Петербург,2020-02-10,4599.0
2019-08-18,KupiBilet.ru,Москва,Санкт-Петербург,2019-09-05,4791.0
2019-08-19,Pobeda,Москва,Санкт-Петербург,2019-11-09,4499.0
2019-08-20,Pobeda,Москва,Санкт-Петербург,2019-11-21,4499.0
2019-08-21,Aviakassa,Москва,Санкт-Петербург,2019-11-22,5080.0


**2). В приложении парсинга википедии получить первую ссылку из раздела "Ссылки" и вывести все значимые слова из неё. Результат записать в файл в форматированном виде.**

**3).* Научить приложение определять количество ссылок в статье (раздел Ссылки). Выполнить поиск слов в статьях по каждой ссылке и результаты записать в отдельные файлы.**

Функция получения ссылок через API:

In [5]:
def api_wikiextlinks(topic):
    
    # получаем данные с помощью штатного API    
    s = requests.Session()
    url = 'https://ru.wikipedia.org/w/api.php'
    params = {
        "action": "query",
        "format": "json",
        "titles": topic,
        "prop": "extlinks" # внешние ссылки
    }

    r = s.get(url=url, params=params)
    data = r.json()
    
    # получаем внешние ссылки из json-а
    extlinks = set()
    for d in data['query']['pages'].values():
        for site in d['extlinks']:
            for link in site.values():
                extlinks.add(link)
                
    return extlinks

Функция получения ссылок через парсинг html регулярными выражениями:

In [6]:
def get_topic_page(topic):
    link = 'https://ru.wikipedia.org/wiki/' + topic.capitalize()
    html = requests.get(link).text
    return html

def re_wikiextlinks(topic):
    html_content = get_topic_page(topic)
    re_result = re.findall(u'class="external\stext"\shref="http[/0-9a-zA-Z.%?=;:_&]+"', html_content)
    extlinks = set(map(lambda s: s[28:-1], re_result))
    return extlinks

Найдем количество уникальных внешних ссылок на вики-статье через обе функции:

In [7]:
topic = u'Саломея_(опера)'

In [8]:
extlinks_api = api_wikiextlinks(topic)

len(extlinks_api)

9

In [9]:
extlinks_re = re_wikiextlinks(topic)

len(extlinks_re)

8

Напишем функцию, которая проходит по доступным внешним ссылкам, получает список комбинаций из русских и латинских символов (более чем 3), находит уникальные комбинации-слова и их количества, и записывает в отдельный файл для каждой ссылки:

In [10]:
def get_page(url):
    try:
        r = requests.get(url)
    except requests.exceptions.MissingSchema:
        url = 'http:' + url
        r = get_page(url)
        
    return r

def common_words(extlinks, file_prefix='api'):
    for n, url in enumerate(extlinks):
        try:
            r = get_page(url)
            r_status = r.status_code == requests.codes.ok
        except requests.exceptions.ConnectionError:
            print(u'ConnectionError')
            r_status = False
    
        if r_status:
            print(n + 1, url)
            html_content = r.text
            words = re.findall(u"[а-яА-Яa-zA-Z]{3,}", html_content)
            unique_words, counts = np.unique(words, return_counts=True)
        
            file_name = '{}_{}.pickle'.format(file_prefix, n + 1)
            with open(file_name, 'wb') as f:
                pickle.dump((unique_words, counts), f)    

Доступные ссылки из API:

In [11]:
common_words(extlinks_api)

1 http://www.operaclass.com/catalogo/opera.asp?idioma=&idOpera=235&idCat=oc
2 http://classiclive.org/index.php?option=com_content&view=article&id=166:2011-05-12-09-37-16&catid=37:2011-05-01-13-42-34&Itemid=62
3 https://books.google.com/books?id=lOeor7_DSVUC
4 https://books.google.com/books?id=jUmBd7pwnQkC
5 https://books.google.com/books?id=xdc_KMC5Ok4C
6 http://archives.metoperafamily.org/archives/scripts/cgiip.exe/WService=BibSpeed/fullcit.w?xCID=38600&limit=500&xBranch=ALL&xsdate=&xedate=&theterm=&x=0&xhomepath=&xhome=
7 http://classicalcdreview.com/Salome.html
8 https://books.google.com/books?id=0MFx0rA4tdgC
9 //www.wikidata.org/wiki/Q279272?uselang=ru


Проверим восьмой файл:

In [12]:
with open('api_8.pickle', 'rb') as f:
    data_new = pickle.load(f)

print(data_new)

(array(['AFLRE', 'AFQjCNGrxl', 'AfzrSlNiLWeCQ', 'Alexander',
       'Autobiography', 'BiBTeX', 'Biography', 'Bolero', 'BookList',
       'Books', 'CTvjHENLomVMCGb', 'Cambridge', 'Canadian', 'Centre',
       'Click', 'DAQ', 'DDjX', 'DMB', 'DOCTYPE', 'DVtjN', 'Date', 'Der',
       'Dhttps', 'Dlibrary', 'Dlookup', 'DqYVto', 'Dru', 'ENpmAAAAMAAJ',
       'EndNote', 'English', 'Enigma', 'FHyvp', 'FaqHv', 'Fas', 'Fid',
       'Fop', 'For', 'Fquery', 'GNxrS', 'German', 'Glenn', 'Google',
       'Gould', 'Great', 'Guide', 'Guy', 'HBJMku', 'HRhWNIh', 'Host',
       'ISBN', 'Information', 'InitNavbar', 'Introducing',
       'IsBooksRentalEnabled', 'IsBooksUnifiedLeftNavEnabled',
       'IsBrowsingHistoryEnabled', 'IsZipitFolderCollectionEnabled',
       'Jewish', 'Jxe', 'KKWaan', 'Kennedy',
       'KlbIPqmAIQrmdDWjjYIFaCnyiEqW', 'Kometentanz', 'Lauded', 'Login',
       'London', 'MFx', 'Man', 'Marco', 'Michael', 'Milnes', 'Musician',
       'Napoleon', 'Nazi', 'Nietzschean', 'Opera', 'Ozon', 'Pl

Доступные ссылки из результатов парсинга:

In [13]:
common_words(extlinks_re, 're')

2 https://books.google.com/books?id=jUmBd7pwnQkC
3 https://books.google.com/books?id=xdc_KMC5Ok4C
4 https://www.wikidata.org/wiki/Q279272?uselang=ru
5 http://classicalcdreview.com/Salome.html
6 http://archives.metoperafamily.org/archives/scripts/cgiip.exe/WService=BibSpeed/fullcit.w?xCID=38600&amp;limit=500&amp;xBranch=ALL&amp;xsdate=&amp;xedate=&amp;theterm=&amp;x=0&amp;xhomepath=&amp;xhome=
7 https://books.google.com/books?id=0MFx0rA4tdgC
8 https://books.google.com/books?id=lOeor7_DSVUC


Проверим первый файл:

In [14]:
with open('re_1.pickle', 'rb') as f:
    data_new = pickle.load(f)

print(data_new)

(array(['AAACA', 'AAB', 'Andreykor', 'Anne', 'Atom', 'BBDC', 'Blocked',
       'BonJhony', 'Bpage', 'ButtonInputWidget', 'CEB',
       'CentralAutoLogin', 'CentralNotice', 'Cext',
       'CheckboxMultiselectInputWidget', 'Cjquery', 'Cmediawiki',
       'Content', 'Coojs', 'Cookie', 'Cshared', 'Cskins', 'Cstyles',
       'DOCTYPE', 'DateInputWidget', 'DimaBot', 'Dodonov',
       'DropdownInputWidget', 'EACBE', 'EditURI', 'FAEBA', 'Fest',
       'FieldLayout', 'Foundation', 'FundraiserRedirector', 'Grig',
       'HTMLDateTimeField', 'HTMLMultiSelectField', 'HTMLSelectField',
       'HTMLTagFilter', 'HTMLTitleTextField', 'HTMLUserTextField', 'Help',
       'How', 'IluvatarBot', 'Kyiv', 'Log', 'Matiz', 'MediaWiki',
       'MyLanguage', 'Neogobius', 'PHP', 'Passionism', 'Powered',
       'Privacy', 'Puzin', 'QBA', 'RLCONF', 'RLPAGEMODULES', 'RLQ',
       'RLSTATE', 'Redirect', 'ResourceLoaderDynamicStyles', 'Rfszabirov',
       'Roman', 'Rumble', 'Sergeykuklin', 'Special', 'Sussapiens',
   